# 2.1: Building a language model

`kaldi` has native support for the `ARPA` format for language models.  A good explanation of that format can be read [here](https://cmusphinx.github.io/wiki/arpaformat/), but here is the basic format:

A popular open-source language modeling toolkit that outputs in the `ARPA` format is `IRSTLM`.  It's manual can be found in `resource_files/irstlm-manual.pdf`.

We will build a language model from a toy corpus (using `IRSTLM`) and then analyze it.

## the toy corpus

A toy corpus is in `resource_files/animal_corpus.txt`.  In this corpus, each line represents a sentence, and there is *no* punctuation present.

**Note:** From the perspective of a language model, one *could* model punctuation if that were of importance, but since our purpose is to model *spoken* text, we do *not* have any need to model punctuation.

In [1]:
cat resource_files/animal_corpus.txt

the mouse ate the cheese
the cat ate the mouse
the dog ate the cat
the lion at the dog
the tyrannosaurus rex ate the lion
the human shot the mouse
the human shot the cat
the human shot the dog
the human shot the lion
the human shot the tyrannosaurus rex


## building the language model with `IRSTLM`

After `export`ing a few variables, we will be able to call scripts from `IRSTLM` without a full path.

In [2]:
export IRSTLM=${KALDI_PATH}/tools/irstlm
export PATH=${PATH}:${IRSTLM}/bin

### `add-start-end.sh`

Since our corpus does *not* have periods, we need to add a custom symbol to represent the *beginning* and *end* of each sentence.

In [4]:
add-start-end.sh -h


add-start-end.sh - adds sentence start/end symbols in each line and trims very very long words

USAGE:
       add-start-end.sh [options]

OPTIONS:
       -h        Show this message
       -r count  Specify symbol repetitions (default 1)
       -t length Trim words up to _length_ chars (default 80)
       -s char   Specify symbol (default s)



In [22]:
cat resource_files/animal_corpus.txt | add-start-end.sh > resource_files/animal_corpus_start_stop.txt

In [24]:
cat resource_files/animal_corpus_start_stop.txt

<s> the mouse ate the cheese  </s>
<s> the cat ate the mouse  </s>
<s> the dog ate the cat  </s>
<s> the lion at the dog  </s>
<s> the tyrannosaurus rex ate the lion  </s>
<s> the human shot the mouse  </s>
<s> the human shot the cat  </s>
<s> the human shot the dog  </s>
<s> the human shot the lion  </s>
<s> the human shot the tyrannosaurus rex  </s>


### `build-lm.sh`

Now let's build the actual language model using `build-lm.sh`

In [7]:
build-lm.sh -h


build-lm.sh - estimates a language model file and saves it in intermediate ARPA format

USAGE:
       build-lm.sh [options]

OPTIONS:
       -i|--InputFile          Input training file e.g. 'gunzip -c train.gz'
       -o|--OutputFile         Output gzipped LM, e.g. lm.gz
       -k|--Parts              Number of splits (default 5)
       -n|--NgramSize          Order of language model (default 3)
       -d|--Dictionary         Define subdictionary for n-grams (optional, default is without any subdictionary)
       -s|--LanguageModelType  Smoothing methods: witten-bell (default), shift-beta, improved-shift-beta, stupid-backoff; kneser-ney and improved-kneser-ney still accepted for back-compatibility, but mapped into shift-beta and improved-shift-beta, respectively
       -p|--PruneSingletons    Prune singleton n-grams (default false)
       -f|--PruneFrequencyThreshold      Pruning frequency threshold for each level; comma-separated list of values; (default is '0,0,...,0', for all level

The main arguments we will focus on are:
 - `-i`
 - `-o`
 - `-n`

`-k` is an important argument for efficient language modeling on a very large corpus.  With our toy example, we do not need to worry about that.  You'll also notice a number of options for `-s` which relate to the type of `smoothing` used.  Stanford has a great resource on `smoothing` that you can find in `resource_files/smoothing_explained.pdf`.

In [40]:
build-lm.sh \
    -i resource_files/lm_temp/animal_corpus_start_stop.txt \
    -o resource_files/animal_lm-2_gram.lm \
    -n 2

LOGFILE:/dev/null
$bin/ngt -i="$inpfile" -n=$order -gooout=y -o="$gzip -c > $tmpdir/ngram.${sdict}.gz" -fd="$tmpdir/$sdict" $dictionary $additional_parameters >> $logfile 2>&1
$scr/build-sublm.pl $verbose $prune $prune_thr_str $smoothing "$additional_smoothing_parameters" --size $order --ngrams "$gunzip -c $tmpdir/ngram.${sdict}.gz" -sublm $tmpdir/lm.$sdict $additional_parameters >> $logfile 2>&1


`IRSTLM` automatically `compresses` the resulting language model.  So we will `decompress` it so we can look at it.

In [42]:
gzip -d resource_files/animal_lm-2_gram.lm.gz

In [44]:
cat resource_files/animal_lm-2_gram.lm

iARPA

\data\
ngram 1=	15
ngram 2=	25

\1-grams:
-1.662758	<s>	-0.845098
-0.641569	the	-0.586266
-1.361728	mouse	-0.397940
-1.264818	ate	-0.698970
-1.662758	cheese	-0.301030
-0.922395	</s>	-1.041393
-1.361728	cat	-0.397940
-1.361728	dog	-0.397940
-1.361728	lion	-0.397940
-1.662758	at	-0.301030
-1.486667	tyrannosaurus	-0.477121
-1.486667	rex	-0.301030
-1.185637	human	-0.778151
-1.185637	shot	-0.778151
-0.787697 <unk>
\2-grams:
-0.845098	<s> <s>
-0.146128	<s> the
-0.954243	the mouse
-1.431364	the cheese
-0.954243	the cat
-0.954243	the dog
-0.954243	the lion
-1.130334	the tyrannosaurus
-0.732394	the human
-0.698970	mouse ate
-0.397940	mouse </s>
-0.096910	ate the
-0.301030	cheese </s>
-0.698970	cat ate
-0.397940	cat </s>
-0.698970	dog ate
-0.397940	dog </s>
-0.397940	lion </s>
-0.698970	lion at
-0.301030	at the
-0.176091	tyrannosaurus rex
-0.602060	rex ate
-0.602060	rex </s>
-0.079181	human shot
-0.079181	shot the
\end\


### `compile-lm`

`compile-lm` can be used to put our language model into memory and evaluate in it in different ways.

In [13]:
compile-lm


compile-lm - compiles an ARPA format LM into an IRSTLM format one

USAGE:
       compile-lm [options] <input-file.lm> [output-file.blm]

DESCRIPTION:
       compile-lm reads a standard LM file in ARPA format and produces
       a compiled representation that the IRST LM toolkit can quickly
       read and process. LM file can be compressed.

OPTIONS:
Parameters:
    Help:      print this help
    d:      verbose output for --eval option; default is 0
    debug:      verbose output for --eval option; default is 0
    dict_load_factor:      sets the load factor for ngram cache; it should be a positive real value; default is 0
    dub:      dictionary upperbound to compute OOV word penalty: default 10^7
    e:      computes perplexity of the specified text file
    eval:      computes perplexity of the specified text file
    f:      filter a binary language model with a word list
    filter:      filter a binary language model with a word list
    h:      print this help
    i:      bui

So we have a second corpus that we will consider our "test" corpus.

In [25]:
cat resource_files/animal_test_corpus.txt

<s> the mouse ate the cat </s>
<s> the tyrannosaurus shot the human </s>
<s> the triceratops ate the lettuce </s>


Note that this test corpus already has the `start` and `stop` symbls added.

#### `perplexity`

In [28]:
compile-lm resource_files/animal_lm-2_gram.lm --eval=yes < resource_files/animal_test_corpus.txt

inpfile: resource_files/animal_lm-2_gram.lm
outfile: animal_lm-2_gram.lm.blm
evalfile: yes
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 14
OOV code is 14
Start Eval
OOV code: 14
%% Nw=0 PP=-nan PPwp=-nan Nbo=0 Noov=0 OOV=-nan%


#### `probability` of `n-gram`s

In [31]:
compile-lm resource_files/animal_lm-2_gram.lm --score=yes < resource_files/animal_test_corpus.txt

inpfile: resource_files/animal_lm-2_gram.lm
outfile: animal_lm-2_gram.lm.blm
interactive: 1
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 14
OOV code is 14
> <s> the	1 p= -0x1.2ad5dc53d7533p-2 bo= 0
> the mouse	1 p= -0x1.0ce06c02e8a4cp+1 bo= 0
> mouse ate	1 p= -0x1.8199d8c0e0c22p+0 bo= 0
> ate the	1 p= -0x1.57576708db605p-3 bo= 0
> the cat	1 p= -0x1.0ce06c02e8a4cp+1 bo= 0
> cat </s>	1 p= -0x1.9b50bb08ac0a2p-1 bo= 0
> <s> the	1 p= -0x1.2ad5dc53d7533p-2 bo= 0
> the tyrannosaurus	1 p= -0x1.3f4f94250d429p+1 bo= 0
> tyrannosaurus shot	1 p= -0x1.ea10eef56f1fep+1 bo= 1
> shot the	1 p= -0x1.19f81ab71a53dp-3 bo= 0
> the human	1 p= -0x1.9959c279b8b8ep+0 bo= 0
> human </s>	1 p= -0x1.f53413f41718ap+1 bo= 1
> <s> the	1 p= -0x1.2ad5dc53d7533p-2 bo= 0
> the <unk>	1 p= -0x1.348217af7430ap+4 bo= 1
> <unk> ate	1 p= -0x1.74c7ea98d13adp+1 bo= 1
> ate the	1 p= -0x1.57576708db605p-3 bo= 0
> the <unk>	1 p= -0x1.348217af7430ap+4 bo= 1
> <unk> </s>	1 p= -0x1.0fdbb970ffb3p+1 bo= 1
> 


In [30]:
compile-lm resource_files/animal_lm-2_gram.lm --sentence=yes < resource_files/animal_test_corpus.txt

inpfile: resource_files/animal_lm-2_gram.lm
outfile: animal_lm-2_gram.lm.blm
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 14
OOV code is 14
Saving in bin format to animal_lm-2_gram.lm.blm


In [34]:
echo "the human ate the cheese" | compile-lm resource_files/animal_lm-2_gram.lm --ngramscore=yes 

inpfile: resource_files/animal_lm-2_gram.lm
outfile: animal_lm-2_gram.lm.blm
interactive for ngrams only: 1
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 14
OOV code is 14
